In [15]:
import os

import einops
import numpy as np
import torch as t
import torch.nn as nn
import torchrl
import wandb
import functools
from functools import partial
from fancy_einsum import einsum

from eindex import eindex
from jaxtyping import Float, Int
from torch import Tensor
from torchtyping import TensorType as TT

import circuitsvis as cv
import transformers
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from transformer_lens.hook_points import (HookedRootModule, HookPoint)  # Hooking utilities
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel, AutoModelForCausalLM
from torchrl.envs.llm import ChatEnv
from torchrl.modules.llm import TransformersWrapper
from tqdm import tqdm

os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = t.device("cuda" if t.cuda.is_available() else "mps" if t.backends.mps.is_available() else "cpu")
print(f"Device: {device}")

Device: mps


In [ ]:
# load artifact from wandb
run = wandb.init()
artifact = run.use_artifact('djdumpling-yale/rlhf_transformers/model-gpt2_20250730-004324:v0', type='model')
model_dir = artifact.download()

# set location locally 
model_path = os.path.join(model_dir, "final_model.pt")
state_dict = t.load(model_path, map_location=device)

# get weights from RLHFed model
base_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('base_model.'):
        new_key = k.replace('base_model.', '')
        base_state_dict[new_key] = v

# load into gpt architecture
source_model = AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb")
base_model = HookedTransformer.from_pretrained("gpt2", device=device, hf_model = source_model)
base_model.eval()

rlhf_model = HookedTransformer.from_pretrained("gpt2", device=device)
rlhf_model.load_state_dict(base_state_dict)
rlhf_model.eval()

# if TRL comes in later?
model = TransformersWrapper(model=rlhf_model, tokenizer=rlhf_model.tokenizer,input_mode="text")

wandb: Currently logged in as: djdumpling (djdumpling-yale) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-gpt2_20250730-004324:v0, 643.08MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:3.5 (183.3MB/s)


config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loaded pretrained model gpt2 into HookedTransformer
Loaded pretrained model gpt2 into HookedTransformer


Interestingly, it seems like the `base_model` already has a preference towards positive movie reviews.

In [3]:
prompt = "This movie was really"

print(rlhf_model.generate(prompt, max_new_tokens=50, temperature=0.7))
print(base_model.generate(prompt, max_new_tokens=50, temperature=0.7))

  0%|          | 0/50 [00:00<?, ?it/s]

This movie was really really very good, and very well done. The music is great and the motion is great, and they are very friendly. It was so addicting and so very exciting. This is one of the best movies I've seen in my life, and


  0%|          | 0/50 [00:00<?, ?it/s]

This movie was really bad. I don't think there were many redeeming qualities in watching this movie. The main character is really annoying; he is not good or interesting. He is annoying because he is a jerk who is going to tell you his life story, not


This is reinforced when looking at the logits, showing that even the `base_model` has a preference towards positive reviews.

In [4]:
utils.test_prompt(prompt, "good", rlhf_model, prepend_bos = True)

Tokenized prompt: ['<|endoftext|>', 'This', ' movie', ' was', ' really']
Tokenized answer: [' good']


Performance on answer token:
Rank: 2        Logit: 15.09 Prob:  5.58% Token: | good|

Top 0th token. Logit: 15.45 Prob:  8.04% Token: | great|
Top 1th token. Logit: 15.13 Prob:  5.85% Token: | amazing|
Top 2th token. Logit: 15.09 Prob:  5.58% Token: | good|
Top 3th token. Logit: 14.89 Prob:  4.57% Token: | fun|
Top 4th token. Logit: 14.41 Prob:  2.83% Token: | awesome|
Top 5th token. Logit: 14.26 Prob:  2.45% Token: | nice|
Top 6th token. Logit: 13.88 Prob:  1.68% Token: | a|
Top 7th token. Logit: 13.85 Prob:  1.62% Token: | really|
Top 8th token. Logit: 13.72 Prob:  1.43% Token: | well|
Top 9th token. Logit: 13.71 Prob:  1.41% Token: | hard|


Ranks of the answer tokens: [(' good', 2)]

In [5]:
utils.test_prompt(prompt, "good", base_model, prepend_bos = True)

Tokenized prompt: ['<|endoftext|>', 'This', ' movie', ' was', ' really']
Tokenized answer: [' good']


Performance on answer token:
Rank: 0        Logit: 17.39 Prob: 12.19% Token: | good|

Top 0th token. Logit: 17.39 Prob: 12.19% Token: | good|
Top 1th token. Logit: 17.30 Prob: 11.15% Token: | bad|
Top 2th token. Logit: 16.20 Prob:  3.69% Token: | funny|
Top 3th token. Logit: 16.18 Prob:  3.61% Token: | great|
Top 4th token. Logit: 16.06 Prob:  3.21% Token: | a|
Top 5th token. Logit: 15.95 Prob:  2.88% Token: | fun|
Top 6th token. Logit: 15.87 Prob:  2.65% Token: | awful|
Top 7th token. Logit: 15.57 Prob:  1.96% Token: | well|
Top 8th token. Logit: 15.38 Prob:  1.63% Token: | terrible|
Top 9th token. Logit: 15.34 Prob:  1.56% Token: | disappointing|


Ranks of the answer tokens: [(' good', 0)]

In [6]:
prompts = ["This movie is really"]
answers = [(" good", " bad")]

answer_tokens = []
for i in range(len(prompts)):
    answer_tokens.append((rlhf_model.to_single_token(answers[i][0]), rlhf_model.to_single_token(answers[i][1])))

answer_tokens = t.tensor(answer_tokens).to(device)

print(answer_tokens)

tensor([[ 922, 2089]], device='mps:0')


In [7]:
tokens = rlhf_model.to_tokens(prompts, prepend_bos = True)

# base_logits.shape: [1,5,50257] --> # prompts, sequence length including prepend_bos , vocabulary size
base_logits, base_cache = base_model.run_with_cache(tokens)
rlhf_logits, rlhf_cache = rlhf_model.run_with_cache(tokens)

In [14]:
def logit_diff(logits, answer_tokens):
    final_logit = logits[:, -1, :]
    answer_logit = final_logit.gather(dim = -1, index = answer_tokens)
    answer_logit_diff = answer_logit[:, 0] - answer_logit[:, 1]

    return answer_logit_diff

logit_diff_base = logit_diff(base_logits, answer_tokens).item()
logit_diff_rlhf = logit_diff(rlhf_logits, answer_tokens).item()
print(f"Logit diff for base model: {logit_diff_base:.3f}")
print(f"Logit diff for rlhf model: {logit_diff_rlhf:.3f}")

Logit diff for base model: -0.185
Logit diff for rlhf model: 1.851


In [ ]:
# answer_residual_directions shape: [1, 2, 768] --> prompt, # tokens in answer_tokens, hidden size of model
answer_residual_directions = rlhf_model.tokens_to_residual_directions(answer_tokens)
logit_diff_directions = answer_residual_directions[:, 0, :] - answer_residual_directions[:, 1, :]

torch.Size([1, 768])

In [ ]:
# cache syntax: [activation_name, layer_index, sub_layer_type]
# final_residual_stream_xxx shape: [1, 5, 768]
final_residual_stream_base = base_cache["resid_post", -1]
final_residual_stream_rlhf = rlhf_cache["resid_post", -1]

# final_token_residual_stream_base_xxxx shape: [1, 768]
final_token_residual_stream_base = final_residual_stream_base[:, -1, :]
final_token_residual_stream_rlhf = final_residual_stream_rlhf[:, -1, :]

# layernorm scaling so that the contribution at each layer is consistent across the network
scaled_final_token_residual_stream_base = base_cache.apply_ln_to_stack(final_token_residual_stream_base, layer = -1, pos_slice = -1)
scaled_final_token_residual_stream_rlhf = rlhf_cache.apply_ln_to_stack(final_token_residual_stream_rlhf, layer = -1, pos_slice = -1)

ln_logit_diff_base = einsum("batch d_model, batch d_model -> ", scaled_final_token_residual_stream_base, logit_diff_directions)
ln_logit_diff_rlhf = einsum("batch d_model, batch d_model -> ", scaled_final_token_residual_stream_rlhf, logit_diff_directions)


torch.Size([1, 768])